In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from xgboost import XGBRegressor
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")

In [ ]:
df_train["kfold"] = -1

In [ ]:
df_train.head()

In [ ]:
kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=df_train)):
    df_train.loc[valid_indicies, "kfold"] = fold

In [ ]:
df_train.head()

In [ ]:
!pip install autoviz
!pip install xlrd

from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()

clear_output()

In [ ]:
sample_submission = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

In [ ]:
ff = AV.AutoViz('../input/30-days-of-ml/train.csv')

In [ ]:
# from xgboost import XGBRegressor
# xgb_params = {
#     'n_estimators': 5000,
#     'learning_rate': 0.12,
#     'subsample': 0.96,
#     'colsample_bytree': 0.12,
#     'max_depth': 2,
#     'booster': 'gbtree', 
#     'reg_lambda': 65.1,
#     'reg_alpha': 15.9,
#     'random_state':40
# }

In [ ]:
df = df_train
df_test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "loss", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

In [ ]:
final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.loss
    yvalid = xvalid.loss
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    params ={'learning_rate': 0.017310085678774113,
 'reg_lambda': 2.05572797087686e-08,
 'reg_alpha': 0.41179423148132366,
 'subsample': 0.8055160198065933,
 'colsample_bytree': 0.4289106022447653,
 'max_depth': 4}
    
    model = XGBRegressor(
        random_state=0, 
        tree_method='gpu_hist',
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=5000,
        **params
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.loss = preds
sample_submission.to_csv("xgb_submission_k", index=False)